# Notebook to demo function calling with Pydantic response model.

In [1]:
import os
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)

## Import needed modules.

In [2]:
from faker import Faker
from gait import MAO, Agent, Dialog, ObserverLoguru
from pydantic import BaseModel, Field
from rich.pretty import pprint

In [3]:
# litellm._turn_on_debug()  # 👈 this is the 1-line change you need to make

## Create a faker instance.

In [4]:
fake = Faker()

## Define functions that the agent will use.

**Make sure to docstring the functions**

In [5]:
def calculate_percentage(value: float, percentage: float) -> float:
    """Calculate what percentage of a value is.

    :param value: The value to calculate the percentage of.
    :param percentage: The percent value. Has to be betwee 0 and 100.
    :return: The percentage value.
    """
    return value * percentage / 100.0

In [6]:
def subtract(a: float, b: float) -> float:
    """Subtract b from a.

    :param a: The value to subtract from.
    :param b: How much to subtract from a.
    :return: The residual.
    """
    return a - b

In [7]:
class TemperatureModel(BaseModel):
    location: str = Field(..., description="The location where to get the temperature.")
    temperature: float = Field(..., description="The temperature at the location.")
    unit: str = Field(..., description="The unit of the temperature.")

In [8]:
def get_temperature_at_location(location: str) -> TemperatureModel:
    """Get temperature in Fahrenheit at a location.

    :param location: The location where to get the temperature. Location can be a place, city, state, zipcode or a country.
    """
    return TemperatureModel(
        location=location,
        temperature=fake.random_int(-5, 40),
        unit="F",
    )

In [9]:
def logger_fn_pprint(call_dict) -> None:
    pprint(call_dict, expand_all=True)

## Define the agent with references to the supporting functions.

In [10]:
agent = Agent(
    # model="ollama_chat/llama3.2:latest",
    # model="ollama_chat/driaforall/tiny-agent-a:3b-q8_0",
    # model="ollama_chat/qwen2.5:7b-instruct-fp16",
    model=f"azure/{os.environ['AZURE_API_DEPLOYMENT']}",
    # model="ollama_chat/granite3.2:8b-instruct-q8_0",
    functions=[
        get_temperature_at_location,
        calculate_percentage,
        subtract,
    ],
    temperature=0.0,
    # logger_fn=logger_fn_pprint,
)

In [11]:
observer = ObserverLoguru()

In [12]:
mao = MAO(agent, observer=observer)
for _ in mao(f"Get the temperature in {fake.city()}"):
    if _.content:
        mao.terminate()

07:18:39 | INFO | GAIT started.
07:18:39 | INFO | Iteration 1 (None) started.
07:18:41 | INFO | Content: None
07:18:41 | INFO | Function: get_temperature_at_location({"location":"West Julieland"})
07:18:41 | INFO | Observation: {"location":"West Julieland","temperature":21.0,"unit":"F"}
07:18:41 | INFO | Iteration 2 (None) started.
07:18:41 | INFO | Content: The temperature in West Julieland is 21°F. If you need more weather details or information about another location, let me know!
07:18:41 | INFO | GAIT ended after 2 iterations.


In [13]:
mao = MAO(agent, observer=observer)
for _ in mao("If I have 15 apples and give away 40%, how many are left?"):
    if _.content:
        mao.terminate()

07:18:53 | INFO | GAIT started.
07:18:53 | INFO | Iteration 1 (None) started.
07:18:54 | INFO | Content: None
07:18:54 | INFO | Function: calculate_percentage({"value":15,"percentage":40})
07:18:54 | INFO | Observation: 6.0
07:18:54 | INFO | Iteration 2 (None) started.
07:18:55 | INFO | Content: None
07:18:55 | INFO | Function: subtract({"a":15,"b":6})
07:18:55 | INFO | Observation: 9
07:18:55 | INFO | Iteration 3 (None) started.
07:18:56 | INFO | Content: If you have 15 apples and give away 40%, you will have 9 apples left.
07:18:56 | INFO | GAIT ended after 3 iterations.
